# SQLite Parameters

In [ ]:
import os
import sqlite3

#### Setup

In [ ]:
# Zet database op
database_file = "sqlite_parameters.db"
connection = sqlite3.connect(database_file)
cursor = connection.cursor()

In [ ]:
# Maak Personen tabel met 2 records
cursor.executescript("""
    CREATE TABLE Personen (
        PersoonId INTEGER PRIMARY KEY,
        Voornaam TEXT NOT NULL,
        Achternaam TEXT NOT NULL
    );

    INSERT INTO Personen
            (Voornaam, Achternaam)
        VALUES
            ("Henk", "Jansen"),
            ("Pieter", "Cornelissen")
    ;
""")

In [ ]:
# Check tabel
cursor.execute("SELECT * FROM Personen").fetchall()

#### Niet geparametriseerde query

In [ ]:
query = "SELECT * FROM Personen WHERE Achternaam = '{achternaam}'"

In [ ]:
# Met correcte invoer...
achternaam = "Jansen"
print(query.format(achternaam=achternaam))
cursor.execute(query.format(achternaam=achternaam)).fetchall()

In [ ]:
# Met incorrecte invoer...
# Merk op dat er teveel informatie wordt teruggegeven!
achternaam = "x' OR 1 = 1 OR 'x"
print(query.format(achternaam=achternaam))
cursor.execute(query.format(achternaam=achternaam)).fetchall()

#### Geparametriseerde query

In [ ]:
# Merk op: De waarde is vervangen door een vraagteken
query = "SELECT * FROM Personen WHERE Achternaam = ?"

In [ ]:
# Met correcte invoer...
# Merk op: Er is 1 vraageken en dus 1 waarde.
achternaam = "Jansen"
cursor.execute(query, [achternaam]).fetchall()

In [ ]:
# Met incorrecte invoer... Geen resultaten!
achternaam = "x' OR 1 = 1 OR 'x"
cursor.execute(query, [achternaam]).fetchall()

In [ ]:
# Mooiere syntax met namen
# Syntax is @naam of :naam, afhankelijk van RDBMS!
query = "SELECT * FROM Personen WHERE Achternaam = @Achternaam"
parameters = {"Achternaam": "Jansen"}

cursor.execute(query, parameters).fetchall()

#### Meerdere rijen tegelijk

In [ ]:
# Data over personen
data = [
    {"Voornaam": "Ingrid", "Achternaam": "Maarsen"},
    {"Voornaam": "Sjoerd", "Achternaam": "van der Poel"},
    {"Voornaam": "Marijke", "Achternaam": "Hoekstra"}
]

In [ ]:
# Query om data in te voeren
query = """
    INSERT INTO Personen (Voornaam, Achternaam)
        VALUES (@Voornaam, @Achternaam)
"""

In [ ]:
# Gerbuik executemany() om alle data in te voeren
cursor.executemany(query, data)

In [ ]:
cursor.execute("SELECT * FROM Personen").fetchall()

In [ ]:
# Afsluiten en opruimen
try:
    connection.close()
    os.remove(database_file)
except FileNotFoundError:
    pass
except PermissionError:
    print(f"Fout: De database {database_file!r} is nog open!")